In [21]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import chardet
import plotly_express as px

""" MOBILITY AND TRANSPORT"""
dataset_path_mat = 'Data/Mobiliteitstrend__per_rit_en_motief_02102024_122208.csv'
with open(dataset_path_mat, 'rb') as rawdata:

    result = chardet.detect(rawdata.read(100000))
    
result

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}

In [22]:
df_mod = pd.read_csv(dataset_path_mat,delimiter=';',encoding='UTF-8-SIG')
df_mod

,Geslacht,Leeftijd,Vervoerwijzen,Reismotieven,Marges,Perioden,Ritten per persoon per dag (gemiddeld) (aantal),Afgelegde afstand per rit (gemiddeld) (km),Reisduur per rit (Minuten)
0,Totaal mannen en vrouwen,Totaal,Totaal,Totaal,Waarde,1999*,3.41,10.96,23.72
1,Totaal mannen en vrouwen,Totaal,Totaal,Totaal,Waarde,2003*,3.30,11.04,23.65
2,Totaal mannen en vrouwen,Totaal,Totaal,Totaal,Waarde,2007*,3.20,11.07,23.88
3,Totaal mannen en vrouwen,Totaal,Totaal,Totaal,Waarde,2011*,3.09,11.21,23.76
4,Totaal mannen en vrouwen,Totaal,Totaal,Totaal,Waarde,2015*,3.03,11.36,24.14
...,...,...,...,...,...,...,...,...,...
1435,Totaal mannen en vrouwen,60 tot 65 jaar,Overige vervoerwijze,Overige reismotieven,Waarde,2019*,.,.,.
1436,Totaal mannen en vrouwen,60 tot 65 jaar,Overige vervoerwijze,Overige reismotieven,Waarde,2020*,.,.,.
1437,Totaal mannen en vrouwen,60 tot 65 jaar,Overige vervoerwijze,Overige reismotieven,Waarde,2021*,.,.,.
1438,Totaal mannen en vrouwen,60 tot 65 jaar,Overige vervoerwijze,Overige reismotieven,Waarde,2022*,.,.,.


In [23]:
df_mod = df_mod.sort_values(['Ritten per persoon per dag (gemiddeld) (aantal)'], ascending=True)
df_mod = df_mod.sort_values(['Perioden'], ascending=True)
df_mod_total = df_mod[
    (df_mod['Geslacht'] == "Totaal mannen en vrouwen") &
    (df_mod["Leeftijd"] == "Totaal") &
    (df_mod["Vervoerwijzen"] != "Totaal")&
    (df_mod["Reismotieven"] == "Totaal")]

df_mod_total['Reismotieven'] = df_mod_total['Vervoerwijzen'].replace({
    "Auto (bestuurder)": "Car (driver)",
    "Overige vervoerwijze": "Other travel mode",
    "Auto (passagier)": "Car (passenger)",
    "Trein": "Train",
    "Lopen": "Walking"})

fig = px.bar(df_mod_total,
        x="Perioden",
        y="Ritten per persoon per dag (gemiddeld) (aantal)",
        color="Reismotieven",
        title= "Travel motives over time",
        labels= {"Perioden": "Periods", "Ritten per persoon per dag (gemiddeld) (aantal)": "Number of trips per day (average)", "Reismotieven": "Travel motives"})
fig.update_yaxes(type='linear')
fig.update_layout(width=1200, height=500)
fig.write_image("Graphs/Travel_modes.png")
fig.show()

C:\Users\Pelle Boucher\AppData\Local\Temp\ipykernel_13244\1290978619.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
df_pct = df_mod_total.copy()
df_pct['Ritten per persoon per dag (gemiddeld) (aantal)'] = (
    df_pct['Ritten per persoon per dag (gemiddeld) (aantal)']
    .str.replace(',', '.')
    .astype(float)  
)

df_pct['percentage'] = df_pct.groupby('Perioden')['Ritten per persoon per dag (gemiddeld) (aantal)'].transform(
    lambda x: (x / x.sum() * 100))


fig = px.bar(df_pct,
        x="Perioden",
        y="percentage",
        color="Vervoerwijzen",
        title="Transportation modes over time",
        labels={"Perioden": "Periods", 
                "percentage": "Percentage of trips (%)", 
                "Vervoerwijzen": "Transportation modes"},
        text_auto='.1f')

fig.update_yaxes(type='linear')
fig.update_layout(width=900, height=900)
fig.write_image("Graphs/Travel_modes_pct.png")
fig.show()